In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## GET INDEKS LINK

In [3]:
url = "http://news.metrotvnews.com/peristiwa/JKRn8m3K-staf-kjri-johor-bahru-belajar-jurnalistik-bersama-metro-tv"

In [7]:
getIndeksLink([], 1, 0, 'news', 'news')

page  1
https://news.metrotvnews.com/index/2018/07/30/0


ConnectionError: HTTPSConnectionPool(host='news.metrotvnews.com', port=443): Max retries exceeded with url: /index/2018/07/30/0 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002209AD8A198>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [6]:
def getIndeksLink(links, page, offset, cat_link, category, date=datetime.strftime(datetime.today(), '%Y/%m/%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
#     if cat_link == 'metrotvnews':
#         url = "https://metrotvnews.com/index/"+date+"/"+ str(offset)
#     else :
    url = "https://"+cat_link+".metrotvnews.com/index/"+date+"/"+ str(offset)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.find('div', class_="style_06").findAll('h2')
    contentDiv
    for post in contentDiv:
        link = [post.find('a',href=True)['href']]
        links.append(link)
            
    el_page = soup.find('div', class_="extrapadding")
    if el_page:
        a_page = el_page.findAll('div', class_='bu fr')[-1].find('a')
        max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
        
        if page < max_page:
            links = getIndeksLink(links, page+1, offset+30, cat_link, category, date)    
        
    return links

## GET DETAIL BERITA

In [ ]:
def getDetailBerita(url):
#     all_articles = []
#     for link in links:
    articles = {}
    #link
#         url = link[0]
    response = requests.get(url)
    html = response.text

    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")

    #extract subcategory from breadcrumb
    bc = soup.find('div', class_="breadcrumbs")
    if not bc:
        continue

    sub = bc.findAll('a')[1].text
    if ("foto" in sub.lower()) or  "video" in sub.lower(): 
        continue

    articles['subcategory'] = sub

    #articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])

    #category
#         articles['category'] = link[1]
    articles['url'] = url

    article = soup.find('div', class_="tru")

    #extract date
    pubdate_author = soup.find("div",class_="reg").text
    pubdate_author_split = pubdate_author.split(' \xa0\xa0 • \xa0\xa0 ')
    pubdate = pubdate_author_split[1]
    pubdate = pubdate.strip(' \t\n\r')
    pubdate = pubdate.replace(' WIB','')
    articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%A, %d %b %Y %H:%M"), "%Y-%m-%d %H:%M:%S")

    #extract author
    author = pubdate_author_split[0]
    articles['author'] = author

    #extract title
    articles['title'] = soup.find('meta', attrs={"property":"og:title"})['content']

    #source
    articles['source'] = 'metrotvnews'

    #extract comments count
    #articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
    articles['comment'] = print("?")
    
    #extract tags
    tags = soup.find('div', class_="line").findAll('a', class_="tag")
    articles['tags'] = ','.join([x.text for x in tags])

    #extract images
    articles['images'] = soup.find('img', class_="pic")['src']

    #extract detail
    detail = soup.find('div', class_="tru")

    #hapus link sisip
    for link in detail.findAll('div', class_="related"):
        link.decompose()

    #hapus video sisip
    for tag in detail.findAll('iframe', class_="embedv"):
        tag.decompose()

    #hapus all setelah clear fix
    #for det in detail.find('div', class_="wfull fl rl"):
    #    det.decompose()

    #hapus all script
    for script in detail.findAll('script'):
        script.decompose()

    #extract content
    detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
    content = re.sub(r'\n|\t|\b|\r','',detail.text)
    articles['content'] = content.strip(' ')
    #print('memasukkan berita id ', articles['id'])
    #all_articles.append(articles)
    return articles

In [ ]:
details = [getDetailBerita(url)]
details

In [ ]:
df = pd.DataFrame(details)

In [ ]:
df